### Import Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from functools import reduce

##### Extract data from json file

In [2]:
a=pd.read_json (r'./DataFiles/AP_BrahmsConcertPart1_File1.json')
a.keypoints = a.keypoints.map(lambda x: x[:3])
b=a[['image_id','keypoints']]

##### Filter x,y, and confidence paramters by converting keypoints

In [3]:
b[['x','y','conf']] = pd.DataFrame(b.keypoints.tolist(), index= b.index)

C:\Users\casap\AppData\Local\Temp\ipykernel_36184\1185701998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b[['x','y','conf']] = pd.DataFrame(b.keypoints.tolist(), index= b.index)
C:\Users\casap\AppData\Local\Temp\ipykernel_36184\1185701998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b[['x','y','conf']] = pd.DataFrame(b.keypoints.tolist(), index= b.index)
C:\Users\casap\AppData\Local\Temp\ipykernel_36184\1185701998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

##### This step is used to create movement limits based on range of motion of each participant. Videos should be analyzed prior to setting the limits.

In [4]:
c1=b[b['x']<165].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c2=b[(b['x']<250)&(b['x']>=165)].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c3=b[(b['x']<400)&(b['x']>=250)].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c4=b[(b['x']<475)&(b['x']>=400)].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c5=b[b['x']>=475].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)

##### Test code to check if database has been split properly

In [ ]:
if(((c1.shape[0]+c2.shape[0]+c3.shape[0]+c4.shape[0]+c5.shape[0])/5)==c1.shape[0]):
    print("Perfect")
else:
    print("Error exists")

## Get only frame number - to be used as an identifier

In [6]:
c1.image_id=c1.image_id.str.split(".",1).str[0].astype(int)
c2.image_id=c2.image_id.str.split(".",1).str[0].astype(int)
c3.image_id=c3.image_id.str.split(".",1).str[0].astype(int)
c4.image_id=c4.image_id.str.split(".",1).str[0].astype(int)
c5.image_id=c5.image_id.str.split(".",1).str[0].astype(int)

## Code to check if there is any missing sequence

In [72]:
seq = pd.RangeIndex(c5.image_id.min(), c5.image_id.max())
seq[~seq.isin(c5.image_id)].values

array([], dtype=int64)

In [73]:
seq = pd.RangeIndex(c4.image_id.min(), c4.image_id.max())
seq[~seq.isin(c4.image_id)].values

array([], dtype=int64)

In [74]:
seq = pd.RangeIndex(c3.image_id.min(), c3.image_id.max())
seq[~seq.isin(c3.image_id)].values

array([], dtype=int64)

In [75]:
seq = pd.RangeIndex(c2.image_id.min(), c2.image_id.max())
seq[~seq.isin(c2.image_id)].values

array([], dtype=int64)

In [76]:
seq = pd.RangeIndex(c1.image_id.min(), c1.image_id.max())
seq[~seq.isin(c1.image_id)].values

array([], dtype=int64)

## Merge all dataframes

In [7]:
cPreFinal=c1.merge(c2.merge(c3.merge(c4.merge(c5,on='image_id',how='inner'),on='image_id',how='inner'),on='image_id',how='inner'),on='image_id',how='inner')
cPreFinal.columns=['image_id','c1_keypoints','c1_x','c1_y','c1_conf'
                  ,'c2_keypoints','c2_x','c2_y','c2_conf'
                  ,'c3_keypoints','c3_x','c3_y','c3_conf'
                  ,'c4_keypoints','c4_x','c4_y','c4_conf'
                  ,'c5_keypoints','c5_x','c5_y','c5_conf']
cFinal=cPreFinal[['image_id','c1_x','c1_y','c2_x','c2_y','c3_x','c3_y','c4_x','c4_y','c5_x','c5_y']].sort_values(by='image_id').reset_index(drop=True)

C:\Users\casap\AppData\Local\Temp\ipykernel_36184\3683076672.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'y_y', 'x_y', 'conf_y', 'keypoints_y'} in the result is deprecated and will raise a MergeError in a future version.
  cPreFinal=c1.merge(c2.merge(c3.merge(c4.merge(c5,on='image_id',how='inner'),on='image_id',how='inner'),on='image_id',how='inner'),on='image_id',how='inner')


## Compute Time, Distance, Velocity, and Acceleration 

### Time

In [8]:
t_interval=0.0333333333333333

In [9]:
for i,rows in cFinal.iterrows():
    if(i==0):
        cFinal.loc[i, 't'] = 0
    else:
        cFinal.loc[i,'t'] = cFinal.loc[i-1,'t'] + t_interval;

# In order to make changes to the total number of rows a multiple of 15 & 3.

### Very important step since for our use case we are testing the start, middle, and end of phrases. If required, this code can be used to trim the dataframe to make it a multiple of 15 & 3

In [15]:
cFinal=cFinal[0:549]

In [16]:
cFinal

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,c5_y,t
0,0,115.195282,218.366531,204.472473,222.705566,350.922516,217.265533,458.661499,211.814560,541.797852,213.232773,0.000000
1,1,114.755905,219.640488,203.437317,221.983002,350.936584,217.110672,457.244049,211.501221,540.319763,214.812668,0.033333
2,2,114.354630,218.721878,203.958145,221.870529,350.785492,217.054306,458.187622,209.169586,537.758301,216.730225,0.066667
3,3,114.560234,218.824707,204.310364,222.105301,350.882629,216.902542,459.209259,209.548096,535.944641,217.899170,0.100000
4,4,115.184525,217.648193,204.625656,222.507294,350.813690,217.328583,459.625763,206.986847,532.380676,217.271835,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...
544,545,122.406731,220.021759,181.548233,223.023514,337.539215,218.758698,464.142761,210.056946,510.607178,214.508163,18.133333
545,546,120.731102,221.109299,181.980667,223.630920,337.374237,219.012054,463.297791,209.573578,514.179382,214.502045,18.166667
546,547,116.968262,220.678528,181.930206,224.004303,336.632751,218.734299,463.395081,209.971832,515.921143,212.802582,18.200000
547,548,120.688469,220.173767,181.210419,225.002396,336.503540,219.004944,463.340454,208.879608,519.962280,212.177780,18.233333


## Create DB

In [ ]:
path_createDB='./BrahmsConcertPart2/KeypointData/BrahmsConcertPart2_File18.csv'

In [1170]:
cFinal.to_csv(path_createDB)